In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from matplotlib import mlab
import string
import csv
%matplotlib inline
from IPython.display import display
pd.set_option('display.max_columns', 500)
import ast
from collections import Counter
from wordcloud import WordCloud
import random
from PIL import Image

# Identify the Key Concepts of the Videos

In [ ]:
# read the video dataset
dataset = pd.read_csv("../data/video_dataset_metadata.csv")

In [ ]:
import sys
import six
import statistics


names = ["videoId","no_videoframes_labels", "videoframes_labels", "no_videoframes_labels_pos", "videoframes_labels_pos",
         "no_videoframes_labels_median", "videoframes_labels_median", 
         "no_videoframes_labels_avg", "videoframes_labels_avg", 
         "no_videoframes_labels_stdev", "videoframes_labels_stdev",
         "no_videoframes_labels_30", "videoframes_labels_30", "no_entities", "entities", 
         "no_entities_noother", "entities_noother", "no_vision", "vision_labels",
         "no_final_concepts", "final_concepts", 
         "final_concepts_counts", "final_concepts_freqs"]

with open("../data/key_concepts_videos/video_concepts_overview.csv", "w" ) as outSentences:
    writer = csv.writer( outSentences )
    writer.writerow(names)

for i in range(len(dataset)):
    videoid = dataset["youtubeLink"].iloc[i].split("=")[-1]
    
    row = []
    row.append(videoid)

    labels = pd.read_csv("../data/labels_video_frames/" + videoid + ".mp4.csv")
    
    uniqueLabels = list(set(labels["label_lemma"]))
    videolabels_no_current = len(list(set(labels["label_lemma"])))
    videolabels_text_current = ("_###_").join(list(set(labels["label_lemma"])))
    
    row.append(videolabels_no_current)
    row.append(videolabels_text_current)
 
    max_duration = dataset['duration'].iloc[i]
    
    part1_vid = max_duration / 3
    part2_vid = part1_vid * 2
    part3_vid = max_duration
    
    vid_part1 = labels[labels["time_offset"] <= part1_vid]
    vid_part2 = labels[(labels["time_offset"] > part1_vid) & (labels["time_offset"] <= part2_vid)]
    vid_part3 = labels[labels["time_offset"] > part2_vid]
    
    pos_labels = []
    pos_times = []
    for label in uniqueLabels:
        if label in list(set(vid_part1["label_lemma"])) and label in list(set(vid_part2["label_lemma"])) and label in list(set(vid_part3["label_lemma"])):
            pos_labels.append(label)
            pos_times.append(len(labels[labels["label_lemma"] == label]))
            
    row.append(len(pos_labels))
    row.append(("###").join(pos_labels))
    
    if len(pos_labels) != 0:
        median_value = statistics.median(pos_times)
        average_value = sum(pos_times) / len(pos_times)
        std = statistics.stdev(pos_times)
    else:
        median_value = 0
        average_value = 0
        std = 0
    
    percentage = dataset['duration'].iloc[i] * 30 / 100
    
    averageLabels = []
    medianLabels = []
    stdevLabels = []
    percentageLabels = []
    for label in uniqueLabels:
        if len(labels[labels["label_lemma"] == label]) > median_value:
            medianLabels.append(label)
        if len(labels[labels["label_lemma"] == label]) > average_value:
            averageLabels.append(label)
        if len(labels[labels["label_lemma"] == label]) > std + average_value:
            stdevLabels.append(label)
        if len(labels[labels["label_lemma"] == label]) > percentage:
            percentageLabels.append(label)
    
    row.append(len(medianLabels))
    row.append(("###").join(medianLabels))

    row.append(len(averageLabels))
    row.append(("###").join(averageLabels))
    
    row.append(len(stdevLabels))
    row.append(("###").join(stdevLabels))
    
    row.append(len(percentageLabels))
    row.append(("###").join(percentageLabels))
    
  
    entities = pd.read_csv("../data/named_entities_subtitles/" + videoid + ".mp4.csv")
        
    videolabels_no_current = len(list(set(entities["entity_lemma"])))
    videolabels_text_current = ("_###_").join(list(set(entities["entity_lemma"])))
    
    row.append(videolabels_no_current)
    row.append(videolabels_text_current)
    
    noother = []
    for label in list(set(entities["entity_lemma"])):
        subset = entities[entities["entity_lemma"] == label]
        if subset["type_v3"].iloc[0] != "OTHER":
            noother.append(label)
    
    row.append(len(noother))
    row.append(("###").join(noother))    
    
    lst3 = [value for value in pos_labels if value in medianLabels] 
    row.append(len(lst3))
    row.append(("###").join(lst3))    
    
    lst4 = set(lst3).union(set(noother)) 
    row.append(len(list(lst4)))
    row.append(("###").join(list(lst4))) 
    
    frequencyE = []
    frequencyL = []
    
    for tag in list(lst4):
        subset = entities[(entities["entity_lemma"] == tag) & (entities["type_v3"] != "OTHER")]
        
        if (len(subset) > 0): 
            frequencyE.append((tag, len(subset)))
        else:
            subset = labels[labels["label_lemma"] == tag]
            frequencyL.append((tag, len(subset)))
    
    frequencyEE = []
    frequencyLL = []
    
    if len(frequencyE) != 0:
        maxe = max(frequencyE,key=lambda x:x[1])
    else:
        maxe = 0
    if len(frequencyL) != 0:
        maxl = max(frequencyL,key=lambda x:x[1])
    else:
        maxl = 0
    
    for tag in list(lst4):
        subset = entities[entities["entity_lemma"] == tag]
        
        if (len(subset) != 0): 
            frequencyEE.append((tag, len(subset)/maxe[1] * 1.0))
        else:
            subset = labels[labels["label_lemma"] == tag]
            frequencyLL.append((tag, len(subset)/maxl[1] * 1.0))
    
    allTuples = frequencyE
    for tup in frequencyL:
        allTuples.append(tup)
        
    row.append(allTuples)
    
    allTuples = frequencyEE
    for tup in frequencyLL:
        allTuples.append(tup)
        
    row.append(allTuples)
    
    with open("../data/key_concepts_videos/video_concepts_overview.csv", "a" ) as outSentences:
        writer = csv.writer( outSentences )
        writer.writerow(row)
    

# Generate Summary Wordcloud Visual Explanations

In [ ]:

from collections import Counter
from wordcloud import WordCloud
import random

def grey_color_func(word, font_size, position, orientation, random_state=None,
                    **kwargs):
    return "hsl(120, 100%%, %d%%)" % random.randint(25, 25)

# the file below contains the key concepts we identified in the original video
selected_tags = pd.read_csv("../data/key_concepts_videos/video_concepts_overview.csv")
types_of_summary = ["___10___aq", "___20___aq", "___10___nq", "___20___nq"]

for j in range(len(selected_tags)):
    videoId = selected_tags["videoId"].iloc[j]
    
    for type_sum in types_of_summary:
        # we read now the concepts from the video summaries
        entity_data = pd.read_csv("../data/concepts_video_summaries/entities_" + videoId + type_sum + ".csv")
        label_data = pd.read_csv("../data/concepts_video_summaries/labels_" + videoId + type_sum + ".csv")
    
        subset = selected_tags[selected_tags["videoId"] == videoId]
        tags_in_subset = subset["final_concepts"].iloc[0].split("###")
                           
        
        word_cloud_dict1 = Counter(list(entity_data["entity_lemma"]))
        for key in word_cloud_dict1:
            word_cloud_dict1[key] /= word_cloud_dict1.most_common(1)[0][1]

        word_cloud_dict2 = Counter(list(label_data["label_lemma"]))
        for key in word_cloud_dict2:
            word_cloud_dict2[key] /= word_cloud_dict2.most_common(1)[0][1]

        for key in word_cloud_dict2:
            if key not in word_cloud_dict1:
                word_cloud_dict1[key] = word_cloud_dict2[key]


        word_cloud_dict = Counter()
        for key in word_cloud_dict1:
            if key in tags_in_subset:
                word_cloud_dict[key] = word_cloud_dict1[key]
                

        wordcloud = WordCloud(width = 1000, height = 500, background_color="white").generate_from_frequencies(word_cloud_dict)

        plt.figure(figsize=(7,5))
        plt.imshow(wordcloud.recolor(color_func=grey_color_func, random_state=3), interpolation="bilinear")
        plt.axis("off") 
        plt.tight_layout()
        plt.savefig("../data/visual_explanations/" + videoId + type_sum + '_summary_wordcloud.png', format='png', dpi=1000, bbox_inches = 'tight')
        plt.show()
        plt.close()
            

# Generate Overlap Wordcloud Visual Explanations

In [ ]:
import ast

from collections import Counter
from wordcloud import WordCloud
import random

def grey_color_func(word, **kwargs):
    if word in overlapping_words:
        color = 'green' 
    else:
        color = 'purple'
    return color

# the file below contains the key concepts we identified in the original video
selected_tags = pd.read_csv("../data/key_concepts_videos/video_concepts_overview.csv")
types_of_summary = ["___10___aq", "___20___aq", "___10___nq", "___20___nq"]

for j in range(len(selected_tags)):
    videoId = selected_tags["videoId"].iloc[j]

    for type_sum in types_of_summary:

        # we read now the concepts from the video summaries
        entity_data = pd.read_csv("../data/concepts_video_summaries/entities_" + videoId + type_sum + ".csv")
        label_data = pd.read_csv("../data/concepts_video_summaries/labels_" + videoId + type_sum + ".csv")
    
        subset = selected_tags[selected_tags["videoId"] == videoId]
        tags_in_subset = subset["final_concepts"].iloc[0].split("###")

        word_cloud_dict1 = Counter(list(entity_data["entity_lemma"]))
        for key in word_cloud_dict1:
            word_cloud_dict1[key] /= word_cloud_dict1.most_common(1)[0][1]

        word_cloud_dict2 = Counter(list(label_data["label_lemma"]))
        for key in word_cloud_dict2:
            word_cloud_dict2[key] /= word_cloud_dict2.most_common(1)[0][1]

        for key in word_cloud_dict2:
            if key not in word_cloud_dict1:
                word_cloud_dict1[key] = word_cloud_dict2[key]

        overlapping_words = []
        for key in word_cloud_dict1:
            if key in tags_in_subset:
                overlapping_words.append(key)

        listOfWords = ast.literal_eval(subset["final_concepts_freqs"].iloc[0])

        word_cloud_dict = Counter()
        for key in listOfWords:
            word_cloud_dict[key[0]] = key[1]

        
        wordcloud = WordCloud(width = 1000, height = 500, background_color="white").generate_from_frequencies(word_cloud_dict)

        plt.figure(figsize=(7,5))
        plt.imshow(wordcloud.recolor(color_func=grey_color_func, random_state=3), interpolation="bilinear")
        plt.axis("off") 
        plt.tight_layout()
        plt.savefig("../data/visual_explanations/" + videoId + type_sum + '_overlap_wordcloud.png', format='png', dpi=1000, bbox_inches = 'tight')
        plt.show()
        plt.close()


# Generate Overlap Fraction Visual Explanation

In [ ]:
import matplotlib.pyplot as plt

def grey_color_func(word, **kwargs):
    if word in overlapping_words:
        color = 'green' 
    else:
        color = 'purple'
    return color

# the file below contains the key concepts we identified in the original video
selected_tags = pd.read_csv("../data/key_concepts_videos/video_concepts_overview.csv")
types_of_summary = ["___10___aq", "___20___aq", "___10___nq", "___20___nq"]

for j in range(len(selected_tags)):
    videoId = selected_tags["videoId"].iloc[j]
    

    for type_sum in types_of_summary:

        # we read now the concepts from the video summaries
        entity_data = pd.read_csv("../data/concepts_video_summaries/entities_" + videoId + type_sum + ".csv")
        label_data = pd.read_csv("../data/concepts_video_summaries/labels_" + videoId + type_sum + ".csv")
    
        subset = selected_tags[selected_tags["videoId"] == videoId]
        tags_in_subset = subset["final_concepts"].iloc[0].split("###")

        word_cloud_dict1 = Counter(list(entity_data["entity_lemma"]))
        for key in word_cloud_dict1:
            word_cloud_dict1[key] /= word_cloud_dict1.most_common(1)[0][1]

        word_cloud_dict2 = Counter(list(label_data["label_lemma"]))
        for key in word_cloud_dict2:
            word_cloud_dict2[key] /= word_cloud_dict2.most_common(1)[0][1]

        for key in word_cloud_dict2:
            if key not in word_cloud_dict1:
                word_cloud_dict1[key] = word_cloud_dict2[key]

        overlapping_words = []
        for key in word_cloud_dict1:
            if key in tags_in_subset:
                overlapping_words.append(key)


        listOfWords = ast.literal_eval(subset["final_concepts_freqs"].iloc[0])

        word_cloud_dict = Counter()
        for key in listOfWords:
            word_cloud_dict[key[0]] = key[1]


        # The slices will be ordered and plotted counter-clockwise.
        sizes = [len(overlapping_words) * 100 / len(listOfWords), 100 - (len(overlapping_words) * 100 / len(listOfWords))]

        labels = [str(round(len(overlapping_words) * 100 / len(listOfWords))) + '% of the video concepts \n are covered by \n the video summary', str(round(100 - (len(overlapping_words) * 100 / len(listOfWords)))) + '% of the video concepts \n are not covered by \n the video summary']
        colors = ['green', 'purple']
        explode = (0, 1)  # explode a slice if required

        fig, ax = plt.subplots(figsize=(6, 3), subplot_kw=dict(aspect="equal"))
        
        wedges, texts = ax.pie(sizes, colors=colors, wedgeprops=dict(width=0.5), startangle=-40)

        bbox_props = dict(boxstyle="square,pad=0.3", fc="w", ec="k", lw=0.72)
        kw = dict(arrowprops=dict(arrowstyle="-"),
                  bbox=bbox_props, zorder=0, va="center")

        for i, p in enumerate(wedges):
            ang = (p.theta2 - p.theta1)/2. + p.theta1
            y = np.sin(np.deg2rad(ang))
            x = np.cos(np.deg2rad(ang))
            horizontalalignment = {-1: "right", 1: "left"}[int(np.sign(x))]
            connectionstyle = "angle,angleA=0,angleB={}".format(ang)
            kw["arrowprops"].update({"connectionstyle": connectionstyle})
            ax.annotate(labels[i], xy=(x, y), xytext=(1.35*np.sign(x), 1.4*y),
                        horizontalalignment=horizontalalignment, **kw)

        plt.tight_layout()
        plt.savefig("../data/visual_explanations/" + videoId + type_sum + '_overlap_fraction.png', format='png', dpi=1000, bbox_inches = 'tight')
        plt.show()
        plt.close()



# Generate Combined Wordcloud and Fraction Visual Explanation

In [ ]:
def get_concat_v(im1, im2):
    dst = Image.new('RGB', (im1.width, im1.height + im2.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (0, im1.height))
    return dst

# the file below contains the key concepts we identified in the original video
selected_tags = pd.read_csv("../data/key_concepts_videos/video_concepts_overview.csv")
types_of_summary = ["___10___aq", "___20___aq", "___10___nq", "___20___nq"]

for j in range(len(selected_tags)):
    videoId = selected_tags["videoId"].iloc[j]
    
    for type_sum in types_of_summary:
        im1 = Image.open("../data/visual_explanations/" + videoId + type_sum + '_overlap_wordcloud.png')
        im2 = Image.open("../data/visual_explanations/" + videoId + type_sum + '_overlap_fraction.png')
        get_concat_v(im1, im2).save("../data/visual_explanations/" + videoId + type_sum + '_combined_wordcloud_and_fraction.png')